# Learning Process Verification

### Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Import required libraries

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from datetime import date
from itertools import product
import os

### Prepare data

In [ ]:
# Retrieve normalisation parameters 

norm_param_df = pd.read_csv('/content/drive/MyDrive/KASHIKO/DATASET/TRG_DATASET_NORM_PARAM.csv')

meanR = norm_param_df.loc[norm_param_df["Dataset"] == "AVG", "meanR"].item()
meanG = norm_param_df.loc[norm_param_df["Dataset"] == "AVG", "meanG"].item()
meanB = norm_param_df.loc[norm_param_df["Dataset"] == "AVG", "meanB"].item()

stdR = norm_param_df.loc[norm_param_df["Dataset"] == "AVG", "stdR"].item()
stdG = norm_param_df.loc[norm_param_df["Dataset"] == "AVG", "stdG"].item()
stdB = norm_param_df.loc[norm_param_df["Dataset"] == "AVG", "stdB"].item()

In [ ]:
dataset = datasets.ImageFolder(
    '/content/drive/MyDrive/KASHIKO/DATASET/TEST_0_FINAL/',
    transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((meanR, meanG, meanB), (stdR, stdG, stdB))
    ])
)
testloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=1,
        shuffle=False,
        num_workers=2,
        drop_last=True)

### Load models

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 12, 5)
        self.bn1 = nn.BatchNorm2d(12)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 24, 5)
        self.bn2 = nn.BatchNorm2d(24)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(24*53*53, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)
    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = x.view(-1,24*53*53)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net1 = Net()
state_dict1 = torch.load('/content/drive/MyDrive/KASHIKO/MODELS/model_2021-05-29_12:16:11_ trg_dataset1 batch_size=100 learning_rate=0.001 scheduler_step_size=5 scheduler_gamma=1 weight_decay=0 epoch_number=11 accuracy=97.6.pth')
net1.load_state_dict(state_dict1)


In [ ]:
net2 = Net()
state_dict2 = torch.load('/content/drive/MyDrive/KASHIKO/MODELS/model_2021-05-29_18:41:53_ trg_dataset2 batch_size=100 learning_rate=0.001 scheduler_step_size=5 scheduler_gamma=1 weight_decay=0 epoch_number=16 accuracy=98.1.pth')
net2.load_state_dict(state_dict2)

In [ ]:
net3 = Net()
state_dict3 = torch.load('/content/drive/MyDrive/KASHIKO/MODELS/model_2021-05-30_08:24:13_ trg_dataset3 batch_size=100 learning_rate=0.001 scheduler_step_size=5 scheduler_gamma=1 weight_decay=0 epoch_number=19 accuracy=98.2.pth')
net3.load_state_dict(state_dict3)

### Test models

In [ ]:
# Test model 1
total_all = 0
correct_all = 0

with torch.no_grad():
    for images, labels in testloader:
        net1.eval()
        out = net1(images)

        _, predicted = torch.max(out.data, 1)

        total_all += labels.size(0)
        correct_all += (predicted == labels).sum().item()

test_accuracy_all = 100 * correct_all / total_all
print(test_accuracy_all)



In [ ]:
# Test model 2
total_all = 0
correct_all = 0

with torch.no_grad():
    for images, labels in testloader:
        net2.eval()
        out = net2(images)

        _, predicted = torch.max(out.data, 1)

        total_all += labels.size(0)
        correct_all += (predicted == labels).sum().item()

test_accuracy_all = 100 * correct_all / total_all
print(test_accuracy_all)

In [ ]:
# Test model 3
total_all = 0
correct_all = 0

with torch.no_grad():
    for images, labels in testloader:
        net3.eval()
        out = net3(images)

        _, predicted = torch.max(out.data, 1)

        total_all += labels.size(0)
        correct_all += (predicted == labels).sum().item()

test_accuracy_all = 100 * correct_all / total_all
print(test_accuracy_all)

### Can accuracy be improved by implementing a voting system (with and without confidence threshold)?

In [ ]:

model_test_param_column_list = ['TRUE','PREDICTED1','CORRECT1','SURE1','PREDICTED2','CORRECT2','SURE2','PREDICTED3','CORRECT3','SURE3','FINAL_PREDICTION']
for threshold in [0,0.7,0.8,0.9,0.95,0.96,0.97,0.98]:
    total_all1 = 0
    correct_all1 = 0
    total_sure1 = 0
    correct_sure1 = 0
    total_all2 = 0
    correct_all2 = 0
    total_sure2 = 0
    correct_sure2 = 0
    total_all3 = 0
    correct_all3 = 0
    total_sure3 = 0
    correct_sure3 = 0
    total_final = 0
    correct_final = 0
    model_test_param_df = pd.DataFrame(columns=model_test_param_column_list)

    m = nn.Softmax(dim=1)
    with torch.no_grad():
        for images, labels in testloader:
            net1.eval()
            out1 = net1(images)        
            net2.eval()
            out2 = net2(images)        
            net3.eval()
            out3 = net3(images)

            _, predicted1 = torch.max(out1.data, 1)
            _, predicted2 = torch.max(out2.data, 1)
            _, predicted3 = torch.max(out3.data, 1)
          
            predicted_soft1 = m(out1)
            predicted_soft2 = m(out2)
            predicted_soft3 = m(out3)
            true = labels.item()

            if np.amax(predicted_soft1.numpy()) > threshold:
                total_sure1 += labels.size(0)
                correct_sure1 += (predicted1 == labels).sum().item()
            total_all1 += labels.size(0)
            correct_all1 += (predicted1 == labels).sum().item()
            prediction1 = predicted1.item()
            if predicted1 == labels:
                correct1 = 1
            else:
                correct1 = 0

            if np.amax(predicted_soft1.numpy()) > threshold:
                sure1 = 1
            else:
                sure1 = 0
          
            if np.amax(predicted_soft2.numpy()) > threshold:
                total_sure2 += labels.size(0)
                correct_sure2 += (predicted2 == labels).sum().item()
            total_all2 += labels.size(0)
            correct_all2 += (predicted2 == labels).sum().item()
            prediction2 = predicted2.item()
            if predicted2 == labels:
                correct2 = 1
            else:
                correct2 = 0

            if np.amax(predicted_soft2.numpy()) > threshold:
                sure2 = 1
            else:
                sure2 = 0

            if np.amax(predicted_soft3.numpy()) > threshold:
                total_sure3 += labels.size(0)
                correct_sure3 += (predicted3 == labels).sum().item()
            total_all3 += labels.size(0)
            correct_all3 += (predicted3 == labels).sum().item()
            prediction3 = predicted3.item()
            if predicted3 == labels:
                correct3 = 1
            else:
                correct3 = 0

            if np.amax(predicted_soft3.numpy()) > threshold:
                sure3 = 1
            else:
                sure3 = 0


            if sure1 + sure2 + sure3 == 3:
                if prediction1 + prediction2 + prediction3 >=2:
                    final_prediction = 1
                else:
                    final_prediction = 0
            elif sure1 + sure2 + sure3 == 2:
                if sure1 == 0:
                    if prediction2 + prediction3 == 2:
                        final_prediction = 1
                    elif prediction2 + prediction3 == 0:
                        final_prediction = 0
                    else:
                        final_prediction = -1
                if sure2 == 0:
                    if prediction1 + prediction3 ==2:
                        final_prediction = 1
                    elif prediction1 + prediction3 ==0:
                        final_prediction = 0
                    else:
                        final_prediction = -1
                if sure3 == 0:
                    if prediction2 + prediction1 ==2:
                        final_prediction = 1
                elif prediction2 + prediction1 ==0:
                    final_prediction = 0
                else:
                    final_prediction = -1
            else:
                final_prediction = -1
          
            if final_prediction != -1:
                total_final += labels.size(0)
                correct_final += (final_prediction == labels).sum().item()


            temp_df = pd.DataFrame([[true, prediction1, correct1, sure1,
                                  prediction2, correct2, sure2,
                                  prediction3, correct3, sure3,
                                  final_prediction]], columns=model_test_param_column_list)
            model_test_param_df = model_test_param_df.append(temp_df)


    test_accuracy_all1 = 100 * correct_all1 / total_all1
    test_accuracy_sure1 = 100 * correct_sure1 / total_sure1
    test_accuracy_all2 = 100 * correct_all2 / total_all2
    test_accuracy_sure2 = 100 * correct_sure2 / total_sure2
    test_accuracy_all3 = 100 * correct_all3 / total_all3
    test_accuracy_sure3 = 100 * correct_sure3 / total_sure3
    test_accuracy_final = 100 * correct_final / total_final
    print("Threshold:", threshold)
    print("Model1:", test_accuracy_all1, test_accuracy_sure1, 100 * total_sure1/total_all1)
    print("Model2:", test_accuracy_all2, test_accuracy_sure2, 100 * total_sure2/total_all2)
    print("Model3:", test_accuracy_all3, test_accuracy_sure3, 100 * total_sure3/total_all3)
    print("Final: ", test_accuracy_final, '                 ', 100 * total_final/total_all3)

model_test_param_df.to_csv('/content/drive/MyDrive/KASHIKO/test_model123.csv')